In [ ]:
!pip install pip==24.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install pytorch_lightning==1.8.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchmetrics to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchmetrics to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.7/798.7 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 34.9 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=bd32002bc2dd202e4635dabd215bb193c788194d355bbc1cb6925c803736907e
  Stored in directory: /

In [ ]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/CIRCULA_23_TRANSF_S.xlsx"
libros = pd.read_excel(path, sheet_name=1, parse_dates=['Fecha de préstamo'])

In [4]:
libros.head()

,Biblioteca de Prestamo,Meson de Circulación,Cod Biblioteca origen Item,Biblioteca origen Item,Cod de Localizacion,Localización Item,Prestamos en Sala,Prestamos Externo,Barcode,MMS Id,...,Fecha de ingreso del Item,Categoría Socio,Identificador Socio,dewey,Dewey_1,Dewey3,Dewey4.1,Dewey4.1 - Copia,Dewey_2,TIPO MATERIAL
0,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DC1,Depósito C1,0,1,29004006168206,991009934269707486,...,1997-12-30 00:00:00,Categoría Biblioteca Virtual,3900488,Co868.5 S45p1,2,868.5 S45p1,868,868.0,860,LIBRO
1,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DD1,Depósito D1,0,1,29004005863906,991013769509707486,...,1997-02-24 00:00:00,Categoría Biblioteca Virtual,3900488,928.61 S45j1,0,928.61 S45j1,928,928.0,920,LIBRO
2,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DD1,Depósito D1,0,1,29004007799355,991008897989707486,...,2000-05-22 00:00:00,Categoría Biblioteca Virtual,3900488,986.251 B82,0,986.251 B82,986,986.0,980,LIBRO
3,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DD1,Depósito D1,0,1,29004017298190,991005656579707486,...,2005-03-22 00:00:00,Categoría Biblioteca Virtual,3900488,928.61 S45v2,0,928.61 S45v2,928,928.0,920,LIBRO
4,Biblioteca Luis Ángel Arango,Préstamo Externo Calle 12 - Solo socios,BLAA,Biblioteca Luis Ángel Arango,DD1,Depósito D1,0,1,29004022727563,991009904169707486,...,2010-12-15 00:00:00,Categoría Biblioteca Virtual,3900488,986.126 E75b,0,986.126 E75b,986,986.0,980,LIBRO


In [5]:
libros.shape

(807075, 30)

### Libros leídos solo una vez

In [6]:
prestamos = libros.groupby('Título')['Prestamos Externo'].sum().reset_index()

In [7]:
libros_unico = prestamos[prestamos['Prestamos Externo'] == 1]
print(libros_unico)

                                                   Título  Prestamos Externo
1       00 anos de arte la historia de la creatividad ...                  1
13                       1 2 3 ¡musica iniciacion musical                  1
17      1 de mayo dia internacional de la clase obrera...                  1
21      10 10 10 10 minutos 10 meses 10 anos una idea ...                  1
23                                   10 anos de historias                  1
...                                                   ...                ...
141848  ¿venganza o perdon un camino hacia la reconcil...                  1
141851  ¿vivimos en democracia analisis conceptual del...                  1
141854  ¿y por que canciones para ninos y nenas y guri...                  1
141862                                ¿y su mama que hace                  1
141866                                     ¿ya soy normal                  1

[50663 rows x 2 columns]


In [8]:
# Eliminar libros únicos de los datos principales
data_filtrada = libros[~libros['Título'].isin(libros_unico['Título'])]

In [9]:
data_filtrada.shape

(713134, 30)

### Usuarios que han leído más de 3 libros

In [10]:
user_read_counts = data_filtrada.groupby('Identificador Socio')['Título'].count().reset_index()
user_read_counts.columns = ['Identificador Socio', 'Libros Leídos']

In [11]:
valid_users = user_read_counts[user_read_counts['Libros Leídos'] > 3]

In [12]:
data_filtrada = data_filtrada[data_filtrada['Identificador Socio'].isin(valid_users['Identificador Socio'])]
print(data_filtrada)

              Biblioteca de Prestamo                     Meson de Circulación  \
0       Biblioteca Luis Ángel Arango  Préstamo Externo Calle 12 - Solo socios   
1       Biblioteca Luis Ángel Arango  Préstamo Externo Calle 12 - Solo socios   
2       Biblioteca Luis Ángel Arango  Préstamo Externo Calle 12 - Solo socios   
3       Biblioteca Luis Ángel Arango  Préstamo Externo Calle 12 - Solo socios   
6       Biblioteca Luis Ángel Arango  Préstamo Externo Calle 12 - Solo socios   
...                              ...                                      ...   
400855                    Casa Gomez                Sala Casa Gómez Campuzano   
618661         Biblioteca de Leticia                     Sala general Leticia   
619582         Biblioteca de Leticia                     Sala general Leticia   
619592         Biblioteca de Leticia                     Sala general Leticia   
619598         Biblioteca de Leticia                     Sala general Leticia   

       Cod Biblioteca orige

In [13]:
data_filtrada.shape

(317990, 30)

In [14]:
data_final = data_filtrada.drop_duplicates(subset=['Identificador Socio', 'Título'])

In [15]:
data_final.shape

(255691, 30)

### Codificación de usuarios y libros

In [16]:
from sklearn.preprocessing import LabelEncoder

# Crear instancias de LabelEncoder
user_encoder = LabelEncoder()
book_encoder = LabelEncoder()

# Codificar usuarios y libros
data_final['User_ID'] = user_encoder.fit_transform(data_final['Identificador Socio'])
data_final['Book_ID'] = book_encoder.fit_transform(data_final['Título'])

C:\Users\danie\AppData\Local\Temp\ipykernel_2844\1351856405.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final['User_ID'] = user_encoder.fit_transform(data_final['Identificador Socio'])
C:\Users\danie\AppData\Local\Temp\ipykernel_2844\1351856405.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final['Book_ID'] = book_encoder.fit_transform(data_final['Título'])


In [17]:
print(data_final[['Identificador Socio', 'User_ID', 'Título', 'Book_ID']].head())

# Verificar el número de usuarios y libros codificados
print(f"Total de usuarios únicos: {len(user_encoder.classes_)}")
print(f"Total de libros únicos: {len(book_encoder.classes_)}")

  Identificador Socio  User_ID                                       Título  \
0             3900488    13402  poesia y prosa con 44 textos sobre el autor   
1             3900488    13402     jose asuncion silva el corazon del poeta   
2             3900488    13402                bucaramanga pasado y presente   
3             3900488    13402                almas en pena chapolas negras   
6             3900488    13402         santander en mis mejores fotografias   

   Book_ID  
0    43015  
1    29958  
2     6624  
3     1833  
6    47801  
Total de usuarios únicos: 20171
Total de libros únicos: 56457


### Motor de recomendación

In [19]:
data_final['rank_latest'] = data_final.groupby(['Identificador Socio'])['Fecha de préstamo'].rank(method='first', ascending=False)

train_ratings = data_final[data_final['rank_latest'] != 1]
test_ratings = data_final[data_final['rank_latest'] == 1]

# drop columns that we no longer need
train_ratings = train_ratings[['User_ID', 'Book_ID', 'Fecha de préstamo']]
test_ratings = test_ratings[['User_ID', 'Book_ID', 'Fecha de préstamo']]

C:\Users\danie\AppData\Local\Temp\ipykernel_2844\2591941273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final['rank_latest'] = data_final.groupby(['Identificador Socio'])['Fecha de préstamo'].rank(method='first', ascending=False)


In [20]:
train_ratings.loc[:, 'interaction'] = 1

In [21]:
train_ratings["interaction"].unique()

array([1], dtype=int64)

In [22]:
# Lista de todos los libros
all_bookIds = train_ratings['Book_ID'].unique()

# Contenedores para datos de entrenamiento
users, books, labels = [], [], []

# Conjunto de interacciones positivas (usuario, libro)
user_item_set = set(zip(train_ratings['User_ID'], train_ratings['Book_ID']))

# Relación negativa:positiva de 4:1
num_negatives = 4

# Generar muestras positivas y negativas
for (u, b) in user_item_set:
    # Agregar la interacción positiva
    users.append(u)
    books.append(b)
    labels.append(1)  # Positivo: usuario interactuó con el libro

    # Agregar muestras negativas
    for _ in range(num_negatives):
        # Seleccionar un libro aleatorio
        negative_book = np.random.choice(all_bookIds)
        # Verificar que el libro no haya sido leído por el usuario
        while (u, negative_book) in user_item_set:
            negative_book = np.random.choice(all_bookIds)
        users.append(u)
        books.append(negative_book)
        labels.append(0)

In [23]:
np.sum(labels)

235520

In [24]:
import torch
from torch.utils.data import Dataset


class BooksTrainDataset(Dataset):
    """Dataset de libros en formato PyTorch para entrenamiento.

    Args:
        ratings (pd.DataFrame): DataFrame que contiene las interacciones de usuarios con libros.
        all_bookIds (list): Lista con todos los IDs únicos de libros.

    """

    def __init__(self, data_final):
        self.users, self.items, self.labels = self.get_dataset(data_final)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, data_final):
        users, items, labels = [], [], []
        user_item_set = set(zip(data_final['User_ID'], data_final['Book_ID']))
        all_book_ids = data_final['Book_ID'].unique()

        num_negatives = 4  # Relación 4:1 de negativos a positivos
        for u, b in user_item_set:
            # Agregar interacciones positivas
            users.append(u)
            items.append(b)
            labels.append(1)

            # Agregar muestras negativas
            for _ in range(num_negatives):
                negative_book = np.random.choice(all_book_ids)
                while (u, negative_book) in user_item_set:
                    negative_book = np.random.choice(all_book_ids)
                users.append(u)
                items.append(negative_book)
                labels.append(0)

        # Convertir a tensores de PyTorch
        return (
            torch.tensor(users, dtype=torch.long),
            torch.tensor(items, dtype=torch.long),
            torch.tensor(labels, dtype=torch.float),
        )

In [ ]:
class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)

        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items or books
            ratings (pd.DataFrame): Dataframe containing the book ratings for training
            all_bookIds (list): List containing all book IDs (train + test)
    """

    def __init__(self, num_users, num_items, ratings, all_bookIds):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=32)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=32)

        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.item_embedding.weight)

        self.fc1 = nn.Linear(in_features=64, out_features=32)  # Incrementa a 128
        self.fc2 = nn.Linear(in_features=32, out_features=16)  # Incrementa la siguiente capa
        self.fc3 = nn.Linear(in_features=16, out_features=8)

        self.dropout1 = nn.Dropout(p=0.2)
        self.dropout2 = nn.Dropout(p=0.3)

        self.output = nn.Linear(in_features=8, out_features=1)

        self.ratings = ratings
        self.all_bookIds = all_bookIds

    def forward(self, user_input, item_input):
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concatenate the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layers
        vector = nn.LeakyReLU()(self.fc1(vector))
        vector = self.dropout1(vector)
        vector = nn.LeakyReLU()(self.fc2(vector))
        vector = self.dropout2(vector)
        vector = nn.LeakyReLU()(self.fc3(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred

    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.1)

In [28]:
from torch.utils.data import DataLoader

# Crear dataset
train_dataset = BooksTrainDataset(data_final)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
num_users = data_final['User_ID'].max() + 1
num_items = data_final['Book_ID'].max() + 1
print(num_users)
print(num_items)

20171
56457


In [ ]:
%%time
model = NCF(num_users, num_items, train_loader, data_final['Book_ID'].unique())

trainer = pl.Trainer(max_epochs=7, gpus=1, reload_dataloaders_every_n_epochs=1,
                     enable_progress_bar=True)

c:\Users\danie\anaconda3\envs\my_env\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


CPU times: total: 15.6 ms
Wall time: 1.61 s


In [31]:
trainer.fit(model, train_loader)

Missing logger folder: c:\Users\danie\Documents\PARCIALFINAL_PY\lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 645 K 
1 | item_embedding | Embedding | 1.8 M 
2 | fc1            | Linear    | 2.1 K 
3 | fc2            | Linear    | 528   
4 | fc3            | Linear    | 136   
5 | dropout1       | Dropout   | 0     
6 | dropout2       | Dropout   | 0     
7 | output         | Linear    | 9     
---------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params
9.819     Total estimated model params size (MB)
c:\Users\danie\anaconda3\envs\my_env\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 whi

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=7` reached.


In [32]:
# User-item pairs for testing
test_user_item_set = set(zip(test_ratings['User_ID'], test_ratings['Book_ID']))

# Dict of all items that are interacted with by each user
user_interacted_items = data_final.groupby('User_ID')['Book_ID'].apply(list).to_dict()

In [33]:
hits = []
for (u,i) in test_user_item_set:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_bookIds) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]

    predicted_labels = np.squeeze(model(torch.tensor([u]*100),
                                        torch.tensor(test_items)).detach().numpy())

    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:20].tolist()]

    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)

print("The Hit Ratio @ 20 is {:.2f}".format(np.average(hits)))

The Hit Ratio @ 20 is 1.00
